# Simulación de caracteristicas de máquina de inducción

Ésta aplicación permite la simulación de características de una máquina de inudcción.

In [2]:
%matplotlib widget

In [3]:
import matplotlib.pyplot as plt
import numpy as np
from core.im import im
import ipywidgets
plt.style.use('presentation.mplstyle')

In [4]:

plt.ioff()
plt.clf()

fig, axes = plt.subplots(ncols=2, figsize=(9,4), frameon=False)
fig.canvas.toolbar_visible = False

mi_1 = im()

n_array = np.linspace(0,1499,1000)
mi_1.set_motor('abb_22kW')
mi_1.update(n_array)


line_T_u  = axes[0].plot(n_array,mi_1.T_u)
line_I_1m = axes[1].plot(n_array,np.abs(mi_1.I_1))

line_T_u_nom = axes[0].plot(n_array,(n_array-n_array) + mi_1.T_u_nom)
line_I_1_nom = axes[1].plot(n_array,(n_array-n_array) + mi_1.I_nom)

line_T_u_state = axes[0].plot(mi_1.n_nom, mi_1.T_u_nom,'o')
line_I_1_state = axes[1].plot(mi_1.n_nom, mi_1.I_nom,'o')

sld_R2 = ipywidgets.FloatSlider(orientation='horizontal',description = u"R rotor:", 
                                value=0.0, min=0.0,max= mi_1.R_2_nominal*10, 
                                step=mi_1.R_2_nominal)

sld_U1 = ipywidgets.FloatSlider(orientation='horizontal',description = u"U:", 
                                value=mi_1.U_n, min=0.0,max= mi_1.U_n*1.2,
                                step=1)

sld_freq = ipywidgets.FloatSlider(orientation='horizontal',description = u"freq:", 
                                value=mi_1.freq, min=0.0,max= mi_1.freq*1.2,
                                 step = 1)

ckbox_escalar = ipywidgets.Checkbox(
    value=False,
    description='CTRL Escalar',
    disabled=False
)


prog_c = ipywidgets.IntProgress(
    value=100,
    min=0,
    max=120,
    step=1,
    description='Carga:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal' 
)

drop_mtype = ipywidgets.Dropdown(
    options=[('ABB 3kW 2pp', 'abb_3kW'),
             ('ABB 22kW 2pp','abb_22kW'), 
             ('ABB 90kW 2pp', 'abb_90kW'),
             ('Rotor bobinado 22 kW 2pp', 'abb_22kWw'),
            ],
    value='abb_22kW',
    description='Motor:',
    disabled=False,
)

btn_reset = ipywidgets.Button(
    description='Reiniciar',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Reiniciar',
    icon=''
)

fig.tight_layout()
axes[0].set_title('Par en función de la velocidad')
axes[1].set_title('Corriente en función de la velocidad');

In [8]:
def initial(change):
    
    mtype = drop_mtype.value
    mi_1.set_motor(mtype)
    n_array = np.linspace(0,1499,500)
    mi_1.update(n_array)

    mi_1.R_2_start_0 =  mi_1.R_2_start
    mi_1.R_2_max_0 =  mi_1.R_2_max
    mi_1.R_2_nominal_0 =  mi_1.R_2_nominal
    
    axes[0].set_ylim([0,np.max(mi_1.T_u)*1.2])
    axes[1].set_ylim([0,np.max(np.abs(mi_1.I_1))*1.2])
    axes[0].set_xlim([0,mi_1.n_1*1.1])
    axes[1].set_xlim([0,mi_1.n_1*1.1])
    line_T_u_nom[0].set_data(n_array,(n_array-n_array) + mi_1.T_u_nom)
    line_I_1_nom[0].set_data(n_array,(n_array-n_array) + mi_1.I_nom)
    
    mi_1.R_2_start_0 =  mi_1.R_2_start
    mi_1.R_2_max_0 =  mi_1.R_2_max
    mi_1.R_2_nominal_0 =  mi_1.R_2_nominal

    axes[0].set_xlim([0,mi_1.n_1*1.1])
    axes[1].set_xlim([0,mi_1.n_1*1.1])

    sld_R2.value = 0
    sld_U1.value = mi_1.U_n
    sld_freq.value = mi_1.freq_nom
    ckbox_escalar.value = False
    
    update(change)
    
def update(change):
    if ckbox_escalar.value == True:
        sld_U1.value = sld_freq.value/50*400.0  
    mi_1.R_2_start =  mi_1.R_2_start_0 + sld_R2.value
    mi_1.R_2_max =  mi_1.R_2_max_0 +sld_R2.value
    mi_1.R_2_nominal =  mi_1.R_2_nominal_0+sld_R2.value
    mi_1.U_1 = sld_U1.value
    mi_1.freq = sld_freq.value
    n_array = np.hstack(([0],np.linspace(sld_freq.value*60/mi_1.N_pp*0.8,sld_freq.value*60/mi_1.N_pp*0.999,1000)))
    mi_1.update(n_array)

    idx_n = np.argmax(mi_1.T_u<mi_1.T_u_nom)
    n_interp = np.interp(n_array[idx_n], n_array, n_array)
    
    T_u_interp = np.interp(n_interp, n_array, mi_1.T_u)
    I_1m_interp = np.interp(n_interp, n_array, np.abs(mi_1.I_1))
    
    line_T_u_state[0].set_data(n_interp,mi_1.T_u[idx_n])
    line_I_1_state[0].set_data(n_interp,I_1m_interp)
       
    c = np.abs(mi_1.I_1[idx_n])/mi_1.I_nom

    prog_c.bar_style = 'success'
    if c>1.1:
        prog_c.bar_style = 'warning'
    if c>1.2:
        prog_c.bar_style = 'danger'
    if c>0 and c<1.5:  
        prog_c.value = 100*c

    n_array = np.linspace(0,1499,200)
    mi_1.update(n_array)
    line_T_u[0].set_data(n_array, mi_1.T_u)
    line_I_1m[0].set_data(n_array,np.abs(mi_1.I_1))
    fig.canvas.draw_idle()
    #fig.canvas.flush_events()

sld_R2.observe(update, names='value')
sld_U1.observe(update, names='value')
sld_freq.observe(update, names='value')
ckbox_escalar.observe(update, names='value')
drop_mtype.observe(initial, names='value')
btn_reset.on_click(initial)

layout_row1 = ipywidgets.HBox([fig.canvas])
layout_row2 = ipywidgets.HBox([drop_mtype,sld_R2,sld_U1])
layout_row3 = ipywidgets.HBox([sld_freq,ckbox_escalar,btn_reset,prog_c])
layout = ipywidgets.VBox([layout_row1,layout_row2,layout_row3])

initial(0)


layout